In [ ]:
import spimage
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Load diffraction pattern

In [ ]:
filename = "/Users/benedikt/data/LCLS/amol3416/pbcv/phasing/r0098/r0098_004_0.h5"
img = spimage.sp_image_read(filename,0)
I = abs(img.image)
M = np.array(img.mask,dtype="bool")
spimage.sp_image_free(img)

# Phasing

In [ ]:
# Phasing parameters
niter_raar = 1000
niter_hio  = 0
niter_er   = 1000
niter_total = niter_raar + niter_hio + niter_er
beta = 0.9
support_radius = 11.0
support_area   = ((support_radius**2)*np.pi) / (256.**2)

# Run phasing
R = spimage.Reconstructor(use_gpu=True)
R.set_intensities(I)
R.set_mask(M)
R.set_number_of_iterations(niter_total)
R.set_number_of_outputs_images(5)
R.set_number_of_outputs_scores(200)
R.set_initial_support(radius=support_radius)
#R.append_support_algorithm("static", number_of_iterations=niter_hio)
#R.append_support_algorithm("threshold", center_image=True, update_period=20,
#                       blur_init=5, blur_final=1, threshold_init=0.3, threshold_final=0.15)
R.append_support_algorithm("area", center_image=True, update_period=20, number_of_iterations=niter_raar+niter_er,
                       blur_init=3, blur_final=1, area_init=support_area, area_final=0.0048)
#R.append_support_algorithm("static",  number_of_iterations=niter_er)
R.append_phasing_algorithm("raar",beta_init=beta, beta_final=beta, number_of_iterations=niter_raar, 
                           constraints=['enforce_real', 'enforce_positivity'])
#R.append_phasing_algorithm("hio", beta_init=beta, beta_final=beta, number_of_iterations=niter_hio,
#                           constraints=['enforce_real', 'enforce_positivity'])
R.append_phasing_algorithm("er",  number_of_iterations=niter_er,
                           constraints=['enforce_real', 'enforce_positivity'])
output = R.reconstruct()
R.clear()

# Iterations

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5)
c = 100
for i in range(5):
    axes[0,i].set_title(output['iteration_index_images'][i])
    axes[0,i].imshow(np.abs(output['real_space'][i])[c:-c,c:-c], cmap='viridis', interpolation='none')
    axes[1,i].imshow(output['support'][i][c:-c,c:-c], cmap='gray', interpolation='none')
    axes[2,i].imshow(np.abs(output['fourier_space'][i])**2, norm=colors.LogNorm(vmin=0.1, vmax=1e5), 
                     cmap='inferno', interpolation='none')
    [axes[j,i].axis('off') for j in range(3)]
plt.subplots_adjust(wspace=0, hspace=-0.5)
plt.show()

# Errors and support size

In [ ]:
plt.figure()
plt.title('Final errors: %.5f / %.5f' %(output['fourier_error'][-1], output['real_error'][-1]))
plt.plot(output['iteration_index_scores'][1:], output['fourier_error'][1:], label='Fourier error')
plt.plot(output['iteration_index_scores'][1:], output['real_error'][1:], label='Real error')
plt.semilogy()
plt.xlabel('Nr. of iterations')
plt.ylabel('Error metric')
plt.legend(frameon=False, loc=3)
plt.show()

In [ ]:
plt.figure()
plt.title('Final support size: %.5f' %output['support_size'][-1])
plt.plot(output['iteration_index_scores'], output['support_size'])
plt.xlabel('Nr. of iterations')
plt.ylabel('Support size')
plt.show()


In [ ]:
print "Final errors: %.5f / %.5f / %.5f" %(output['fourier_error'][-1], output['real_error'][-1], output['support_size'][-1])

# Recovered diffraction intensities

In [ ]:
fig = plt.figure(figsize=(10,3.6))
axl = fig.add_subplot(121)
axr = fig.add_subplot(122)
axl.set_xticks([])
axl.set_yticks([])
axr.set_xticks([])
axr.set_yticks([])
axl.set_title('Diffraction pattern')
axr.set_title('Recovered intensities')
tmp = np.abs(I)*M
tmp[tmp<0.1] = 0.1
im = axl.imshow(tmp, norm=colors.LogNorm(vmin=0.1, vmax=1e5), cmap='inferno', interpolation='none')
axr.imshow(np.abs(output['fourier_space'][-1])**2, norm=colors.LogNorm(vmin=0.1, vmax=1e5), cmap='inferno', interpolation='none')
cb = fig.colorbar(im)
cb.ax.set_ylabel('Nr. of photons / px')
plt.show()

# Support and reconstruction

In [ ]:
fig = plt.figure(figsize=(10,3.6))
axl = fig.add_subplot(121)
axr = fig.add_subplot(122)
axl.set_xticks([])
axl.set_yticks([])
axr.set_xticks([])
axr.set_yticks([])
axl.set_title('Support')
axr.set_title('Reconstruction')
c = 110
im = axr.imshow(np.abs(output['real_space'][-1])[c:-c,c:-c], cmap='viridis', interpolation='none')
dx = 7.7 #[nm / px]
axr.add_patch(plt.Rectangle((2,32),100./7.7,1.2, facecolor='w', lw=0))
axr.text(8.2,31.5, r'100 nm', color='w', weight='bold', fontsize=10, ha='center', va='bottom')
axl.imshow(output['support'][-1][c:-c,c:-c], cmap='gray', interpolation='none')
cb = fig.colorbar(im)
cb.ax.set_ylabel('Electron density [arb. units]')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,3.6))
axl = fig.add_subplot(121)
axr = fig.add_subplot(122)
axl.set_xticks([])
axl.set_yticks([])
axr.set_xticks([])
axr.set_yticks([])
axl.set_title('Support')
axr.set_title('Reconstruction')
c = 110
s = output['support'][-1][c:-c,c:-c]
im = axr.imshow(np.angle(output['real_space'][-1])[c:-c,c:-c], cmap='gnuplot', interpolation='none')
dx = 7.7 #[nm / px]
axr.add_patch(plt.Rectangle((2,32),100./7.7,1.2, facecolor='w', lw=0))
axr.text(8.2,31.5, r'100 nm', color='w', weight='bold', fontsize=10, ha='center', va='bottom')
axl.imshow(s, cmap='gray', interpolation='none')
cb = fig.colorbar(im)
cb.ax.set_ylabel('Electron density [arb. units]')
plt.show()